In [1]:
from __future__ import print_function

import sys
import os
import pandas as pd
import numpy as np
import re
import nltk

from keras.layers import Input, Embedding, LSTM, Dense, Bidirectional
from keras.layers import TimeDistributed
from keras.models import Model, load_model
from keras.layers import SimpleRNN

import warnings
warnings.filterwarnings("ignore")

INPUT_LENGTH = 20
OUTPUT_LENGTH = 20

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['ansfile.txt', 'quesfile.txt', 'finalgot.txt']


In [2]:
fn1 = open('../input/quesfile.txt', 'r')
short_questions = fn1.readlines()
short_questions = [x.lower() for x in short_questions]
fn1.close()

In [3]:
fn2 = open('../input/ansfile.txt', 'r')
short_answers = fn2.readlines()
short_answers = [x.lower() for x in short_answers]
fn2.close()

In [4]:
#choosing number of samples
num_samples = 30000  # Number of samples to train on.
short_questions = short_questions[:num_samples]
short_answers = short_answers[:num_samples]
#tokenizing the qns and answers
short_questions_tok = [nltk.word_tokenize(sent) for sent in short_questions]
short_answers_tok = [nltk.word_tokenize(sent) for sent in short_answers]

In [5]:
#train-validation split
data_size = len(short_questions_tok)

# We will use the first 0-80th %-tile (80%) of data for the training
training_input  = short_questions_tok[:round(data_size*(80/100))]
training_input  = [tr_input[::-1] for tr_input in training_input] #reverseing input seq for better performance
training_output = short_answers_tok[:round(data_size*(80/100))]

# We will use the remaining for validation
validation_input = short_questions_tok[round(data_size*(80/100)):]
validation_input  = [val_input[::-1] for val_input in validation_input] #reverseing input seq for better performance
validation_output = short_answers_tok[round(data_size*(80/100)):]

print('training size', len(training_input))
print('validation size', len(validation_input))

training size 9630
validation size 2407


In [6]:
# Create a dictionary for the frequency of the vocabulary
# Create 
vocab = {}
for question in short_questions_tok:
    for word in question:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

for answer in short_answers_tok:
    for word in answer:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1            

In [7]:
# Remove rare words from the vocabulary.
# We will aim to replace fewer than 5% of words with <UNK>
# You will see this ratio soon.
threshold = 2
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

In [8]:
print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)

Size of total vocab: 9567
Size of vocab we will use: 5841


In [9]:
#we will create dictionaries to provide a unique integer for each word.
WORD_CODE_START = 1
WORD_CODE_PADDING = 0

word_num  = 2 #number 1 is left for WORD_CODE_START for model decoder later
encoding = {}
decoding = {1: 'START'}
for word, count in vocab.items():
    if count >= threshold: #get vocabularies that appear above threshold count
        encoding[word] = word_num 
        decoding[word_num ] = word
        word_num += 1

print("No. of vocab used:", word_num)

No. of vocab used: 5843


In [10]:
#include unknown token for words not in dictionary
decoding[len(encoding)+2] = '<UNK>'
encoding['<UNK>'] = len(encoding)+2

dict_size = word_num+1
dict_size

5844

In [11]:
def transform(encoding, data, vector_size=20):
    transformed_data = np.zeros(shape=(len(data), vector_size))
    for i in range(len(data)):
        for j in range(min(len(data[i]), vector_size)):
            try:
                transformed_data[i][j] = encoding[data[i][j]]
            except:
                transformed_data[i][j] = encoding['<UNK>']
    return transformed_data

In [12]:
#encoding training set
encoded_training_input = transform(
    encoding, training_input, vector_size=INPUT_LENGTH)
encoded_training_output = transform(
    encoding, training_output, vector_size=OUTPUT_LENGTH)

print('encoded_training_input', encoded_training_input.shape)
print('encoded_training_output', encoded_training_output.shape)

encoded_training_input (9630, 20)
encoded_training_output (9630, 20)


In [13]:
validation_input.pop()
#encoding validation set
encoded_validation_input = transform(
    encoding, validation_input, vector_size=INPUT_LENGTH)
encoded_validation_output = transform(
    encoding, validation_output, vector_size=OUTPUT_LENGTH)

print('encoded_validation_input', encoded_validation_input.shape)
print('encoded_validation_output', encoded_validation_output.shape)

encoded_validation_input (2406, 20)
encoded_validation_output (2406, 20)


In [14]:
import tensorflow as tf
tf.keras.backend.clear_session()

encoder_input = Input(shape=(INPUT_LENGTH,))
decoder_input = Input(shape=(OUTPUT_LENGTH,))

In [15]:
encoder = Embedding(dict_size, 128, input_length=INPUT_LENGTH, mask_zero=True)(encoder_input)
encoder = LSTM(512, return_sequences=True, unroll=True)(encoder)
encoder_last = encoder[:,-1,:]

print('encoder', encoder)
print('encoder_last', encoder_last)

decoder = Embedding(dict_size, 128, input_length=OUTPUT_LENGTH, mask_zero=True)(decoder_input)
decoder = LSTM(512, return_sequences=True, unroll=True)(decoder, initial_state=[encoder_last, encoder_last])

print('decoder', decoder)

# For the plain Sequence-to-Sequence, we produced the output from directly from decoder
# output = TimeDistributed(Dense(output_dict_size, activation="softmax"))(decoder)

Instructions for updating:
Colocations handled automatically by placer.
encoder Tensor("lstm_1/transpose_2:0", shape=(?, 20, 512), dtype=float32)
encoder_last Tensor("strided_slice:0", shape=(?, 512), dtype=float32)
decoder Tensor("lstm_2/transpose_2:0", shape=(?, 20, 512), dtype=float32)


In [16]:
from keras.layers import Activation, dot, concatenate

# Equation (7) with 'dot' score from Section 3.1 in the paper.
# Note that we reuse Softmax-activation layer instead of writing tensor calculation
attention = dot([decoder, encoder], axes=[2, 2])
attention = Activation('softmax', name='attention')(attention)
print('attention', attention)

context = dot([attention, encoder], axes=[2,1])
print('context', context)

decoder_combined_context = concatenate([context, decoder])
print('decoder_combined_context', decoder_combined_context)

# Has another weight + tanh layer as described in equation (5) of the paper
output = TimeDistributed(Dense(512, activation="tanh"))(decoder_combined_context)
output = TimeDistributed(Dense(dict_size, activation="softmax"))(output)
print('output', output)

attention Tensor("attention/truediv:0", shape=(?, 20, 20), dtype=float32)
context Tensor("dot_2/MatMul:0", shape=(?, 20, 512), dtype=float32)
decoder_combined_context Tensor("concatenate_1/concat:0", shape=(?, 20, 1024), dtype=float32)
output Tensor("time_distributed_2/Reshape_1:0", shape=(?, 20, 5844), dtype=float32)


In [17]:
model = Model(inputs=[encoder_input, decoder_input], outputs=[output])
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 20, 128)      748032      input_2[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 128)      748032      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LS

In [18]:
training_encoder_input = encoded_training_input
training_decoder_input = np.zeros_like(encoded_training_output)
training_decoder_input[:, 1:] = encoded_training_output[:,:-1]
training_decoder_input[:, 0] = WORD_CODE_START
training_decoder_output = np.eye(dict_size)[encoded_training_output.astype('int')]

validation_encoder_input = encoded_validation_input
validation_decoder_input = np.zeros_like(encoded_validation_output)
validation_decoder_input[:, 1:] = encoded_validation_output[:,:-1]
validation_decoder_input[:, 0] = WORD_CODE_START
validation_decoder_output = np.eye(dict_size)[encoded_validation_output.astype('int')]

In [19]:
model.fit(x=[training_encoder_input, training_decoder_input], y=[training_decoder_output],
          validation_data=([validation_encoder_input, validation_decoder_input], [validation_decoder_output]),
          #validation_split=0.05,
          batch_size=64, epochs=100)

model.save('model_attention.h5')

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 9630 samples, validate on 2406 samples
Epoch 1/100
9630/9630 [==============================] - 36s 4ms/step - loss: 0.0015 - val_loss: 0.0012
Epoch 2/100
9630/9630 [==============================] - 21s 2ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 3/100
9630/9630 [==============================] - 21s 2ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 4/100
9630/9630 [==============================] - 21s 2ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 5/100
9630/9630 [==============================] - 20s 2ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 6/100
9630/9630 [==============================] - 21s 2ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 7/100
9630/9630 [==============================] - 20s 2ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 8/100
9630/9630 [==============================] - 20s 2ms/step - loss: 0.0011 - val_loss: 0

In [20]:
def prediction(raw_input):
    input_tok = [nltk.word_tokenize(raw_input)]
    input_tok = [input_tok[0][::-1]]  #reverseing input seq
    encoder_input = transform(encoding, input_tok, 20)
    decoder_input = np.zeros(shape=(len(encoder_input), OUTPUT_LENGTH))
    decoder_input[:,0] = WORD_CODE_START
    for i in range(1, OUTPUT_LENGTH):
        output = model.predict([encoder_input, decoder_input]).argmax(axis=2)
        decoder_input[:,i] = output[:,i]
    return output

In [21]:

def decode(decoding, vector):
    text = ''
    for i in vector:
        if i == 0:
            break
        text += ' '
        text += decoding[i]
    return text

In [22]:
question=[]
answer=[]
for i in range(20):
    seq_index = np.random.randint(1, len(short_questions))
    output = prediction(short_questions[seq_index])
    print ('Q:', short_questions[seq_index])
    print ('A:', decode(decoding, output[0]))

Q:  i do not know what that is

A:  what is dead may never die
Q:  without you there to rule khaleesi i fear the masters will simply bide their time wait for the invaders to leave and areassert control

A:  you are a good man
Q:  arenlys death was no fault of yours you served him bravely

A:  i am not afraid of them i am not going to run
Q:  it was never our forte golden roses indeed your brother and his new queen thought you would be defending casterly rock

A:  you are a pretty lad the girls would claw each others eyes out to get naked with you
Q:  good point

A:  they say he rides into battle on the back of a giant direwolf they say he can turn into into
Q:  now and always

A:  the prince myrcella
Q:  i have nowhere else to go i have taken her to every healer in braavos i have spent every penny i had she suffers every day of her life i just want it to end

A:  you are a pretty man
Q:  a field in the north

A:  the north the abandoned nightfort night
Q:   the lord has smiled upon me


In [23]:
question=[]
answer=[]
for i in range(20):
    seq_index = np.random.randint(1, len(short_questions))
    output = prediction(short_questions[seq_index])
    question.append(short_questions[seq_index])
    answer.append(decode(decoding, output[0]))

In [24]:
zippedList =  list(zip(question, answer))

data = pd.DataFrame(zippedList, columns = ['Questions','Answers'])
data.to_csv('data.csv', index=False)

In [25]:
#For input in kernel only
#raw_input = input()
#output = prediction(raw_input)
#print (decode(decoding, output[0]))